In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

In [ ]:
from ipywidgets import interact, IntRangeSlider, ToggleButtons

In [ ]:
# A bit of customisation
matplotlib.rcParams['figure.figsize'] = (18.0, 5.0)

# %matplotlib inline
%config InlineBackend.figure_format='retina'

np.set_printoptions(precision=3, suppress=True)  # suppress scientific float notation

In [ ]:
path = r'..'
# path = '.'

# Load data
Load all the files as a single dataset using xarray and its dask capabilities. In this way the files are not loaded in-memory but dask/xarray load them **only** when needed. `mask_and_scale` to True replaces array values equal to `_FillValue` with NA and scale values according to the formula `original_values * scale_factor + add_offset`.


In [ ]:
ds = xr.open_mfdataset(path + '/*.nc', mask_and_scale=True)
ds

# Widget

The above selection method allows as to build custom interactive dashboards with a simple jupyter widgets through `ipywidget`. This process is very is simple. You just need to encapsulate you query in a function and decorate it with [interact](https://ipywidgets.readthedocs.io/en/stable/examples/Using%20Interact.html#Basic-interact) by specifying the desired range. Due to xarray and dask's out of core computation, querying and plotting this dataset is almost instant

In [ ]:
@interact(var=list(ds.data_vars), 
          region=ds.coords['region'].values,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    ds[var].sel(time='{}-{}'.format(year, month), region=region).plot(color='k') # year

In [ ]:
@interact(var=list(ds.data_vars), 
          region=ds.coords['region'].values,
          year_range=IntRangeSlider(min=1980,max=2018),
          timestep=ToggleButtons(options=[('week','w'),('month','m'),('year','a')]),
          continuous_update=False,
         )
def plot_region_month(var, region, year_range,timestep='m'):
    (ds[var].sel(region=region,
                 time=slice(str(year_range[0]), str(year_range[1])))
            .resample(time=timestep).mean()
            .plot(color='k')
    )

Querying works also for a selection of regions

In [ ]:
#Extract unique countries by taking the first two characters of all regions
countries = set(s[0:2] for s in ds.coords['region'].values)

@interact(var=list(ds.data_vars), 
          region=countries,
          month=(1,12),
          year=(1980,2018))
def plot_region_month(var, region, year, month):
    selection = [s for s in ds.coords['region'].values if s[0:2]==region]
    ds[var].sel(time='{}-{}'.format(year, month), region=selection).plot.line(x='time')